In [39]:
import pandas as pd

In [2]:
df_pro = pd.read_excel('individual_profiles3.xlsx')
# profile1: Age (20-25 years) + Gender (Male) + Hobby (Sports) + Chatbot experience (Medium-level)
# profile2: Age (20-25 years) + Gender (Female) + Hobby (Sports) + Chatbot experience (Medium-level)
df_pro
profile1 = df_pro.loc[(df_pro['gender'] == 'Male') & (df_pro['hobby_arts'] == 'arts') & (df_pro['medium_level'] == 'medium_level')]
profile1_ids = profile1['ids'].tolist()
print(profile1_ids)
profile2 = df_pro.loc[(df_pro['gender'] == 'Female') & (df_pro['hobby_arts'] == 'arts') & (df_pro['medium_level'] == 'medium_level')]
profile2_ids = profile2['ids'].tolist()
print(profile2_ids)


[92, 113, 97, 109, 141, 130, 147, 93, 133, 135, 137]
[154, 98, 95, 128]


In [5]:
#for friends
import numpy as np
df_buyer = pd.read_excel('/Users/aayeshaaayesha/Downloads/sentiment_analysis/friend_sentiment.xlsx')
print(df_buyer)
df_buyer = df_buyer.drop(index=0)
df_buyer['UID'] = df_buyer['Unnamed: 0'].astype(int)
df_buyer1 = df_buyer.loc[df_buyer['UID'].isin(profile1_ids)]
#df_buyer1.to_excel('/Users/aayeshaaayesha/Downloads/sentiment_analysis/buyer_profiles/buyer_profile1_gender.xlsx')
df_buyer2 = df_buyer.loc[df_buyer['UID'].isin(profile2_ids)]
#df_buyer2.to_excel('/Users/aayeshaaayesha/Downloads/sentiment_analysis/buyer_profiles/buyer_profile2_gender.xlsx')


   Unnamed: 0            Unnamed: 1  \
0         UID              Username   
1          83               Mustafa   
2          82             Alexander   
3          85               Emirhan   
4          83               Mustafa   
..        ...                   ...   
70        114             Anastasia   
71        131  Florian Haubenberger   
72        135                Yannik   
73        138              Kristian   
74        136                  Joan   

                                          interface1f  \
0   Did you find the 2D (two-dimensional) view of ...   
1   Maybe consider adding more than one picture fo...   
2                                           It was ok   
3   There might be more detailed photo like qualit...   
4   I think adding more pics for each item will be...   
..                                                ...   
70  I want an improved view, where i can see how t...   
71  the view was bad tbh, the pictures were croppe...   
72                

In [40]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


In [41]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [42]:
# buyer profile1
import pandas as pd
df = pd.read_excel('buyer_profile1_gender.xlsx')
#df = df.drop(index=0)
#df = df.sample(n=4)
df

,Unnamed: 0.1,Unnamed: 0,UID,Name,interface1b,interface2b,interface3b,function1b,function2b,accomp1b,accomp2b
0,6,52,147,Maximilian Haller,it is definitely interesting. it allows you to...,"yes, it was almost too big","yes, but the intent should also be shown in th...",yes,yes,yes,Not always and of course it is not enough info...
1,0,10,92,Mehdin,"Yes, it was good to see the final selections i...","It was easy, only one problem is that if one u...","Yes, it was clear what the recommendation is.","Yes, but I would have appreciated if the inten...","Yes, as it constraints the purpose of their su...",I could imagine it to be useful if I actually ...,"Yes, I think having the image additionally mak..."
2,1,27,113,Maximilian Höller,"yes, because it was a good overview on what I ...",definitely! no chance to overlook it.,"difficult. one time I had 1, 1, 3, 3 \nhow sho...",sharing yes,"yes, this is very important in my opinion, bec...",for sure it is useful if you care about their ...,"the image is very important and the intent, as..."
3,8,64,133,Andreas,Yes the feature was okay. What was missing is ...,"Yes this was intuitive, as it is the classic i...",The recommended items by use of the filters fr...,Yes I was.,Yes I did,"Yes maybe a little bit, but in the end I would...",Yes I did.


In [43]:
# buyer profile1
from statistics import mean 
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df.shape[0]):
        text = df.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]
for i in range (3, len(df.columns)):
    x = df.columns[i]
    print(x)
    print(sentiForCol(x))

Name
[0.20082518, 0.03994734, 0.7592275]
interface1b
[0.8420993, 0.017777747, 0.14012295]
interface2b
[0.443105, 0.21140432, 0.34549066]
interface3b
[0.08594054, 0.16824152, 0.7458179]
function1b
[0.27440724, 0.055159558, 0.6704332]
function2b
[0.3610841, 0.078945346, 0.5599706]
accomp1b
[0.39149627, 0.057049036, 0.5514547]
accomp2b
[0.31239602, 0.16111691, 0.52648705]


In [44]:
# buyer profile1
all_sentiment = []
for i in range (3, len(df.columns)):
    x = df.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 2:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 3 or j == 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])

print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.4953431785106659
 interface negative sentiment is 0.0897098034620285
 interface neutral sentiment is 0.4149470329284668
 function positive sentiment is 0.18017388880252838
 function negative sentiment is 0.11170053482055664
 function neutral sentiment is 0.7081255912780762
 accomplish positive sentiment is 0.3549921214580536
 accomplish negative sentiment is 0.09903709590435028
 accomplish neutral sentiment is 0.5459707975387573


In [45]:
# buyer profile2
import pandas as pd
df2 = pd.read_excel('buyer_profile2_gender.xlsx')
#df = df.drop(index=0)
#df2 = df2.sample(n=7)
df2

,Unnamed: 0,UID,Name,interface1b,interface2b,interface3b,function1b,function2b,accomp1b,accomp2b
0,15,154,Kathrin,"yes, I liked the brightness of it","if you know how the system works it was easy, yes",it was very easy to understand as it had a cle...,"yes, sharing the link was easy","yes, as it helped me to make decisions on what...","yes, I would also use it outside of this uni c...","yes, I liked their feedback and I found it hel..."
1,32,98,Taranpreet Kaur,"it was ok, maybe different/several angles woul...","yes, it was easy to use, but it only worked on...",it was easy to understand,"yes, didn't have any problem with it","yes, it was useful","yes, it definitely was",yes it was useful
2,40,95,Fardokhtsadat,"The wish list is nice to have, however I would...",Yes. However it would be nice to have a link t...,The recommended list of item was clear when a ...,Yes. I could easily copy the link and share it...,"Yes, the intend helps to provide some context ...","Yes, I think it is cool to get some feedback f...",Yes.
3,49,128,Laura Maria,Yes,Yes,Yes,Yes,"Yes, very useful",Yes,Yes


In [52]:
#df2.to_excel('/Users/aayeshaaayesha/Downloads/sentiment_analysis/friend_profiles/friend_profile2_age.xlsx')

In [46]:
# buyer profile2
from statistics import mean 
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df2.shape[0]):
        text = df2.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]
for i in range (3, len(df.columns)):
    x = df.columns[i]
    print(sentiForCol(x))

[0.12013835, 0.09921248, 0.7806492]
[0.69912225, 0.039813608, 0.26106414]
[0.46919253, 0.042639658, 0.48816782]
[0.48551136, 0.08814134, 0.42634732]
[0.62980133, 0.030873535, 0.33932513]
[0.687593, 0.009907327, 0.3024997]
[0.62199324, 0.029726762, 0.34827998]
[0.6809978, 0.043361258, 0.27564096]


In [47]:
# buyer profile2
all_sentiment = []
for i in range (3, len(df2.columns)):
    x = df2.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 2:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 3 or j == 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])
print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.551275372505188
 interface negative sentiment is 0.056864868849515915
 interface neutral sentiment is 0.391859769821167
 function positive sentiment is 0.6586971282958984
 function negative sentiment is 0.020390430465340614
 function neutral sentiment is 0.3209124207496643
 accomplish positive sentiment is 0.6514955163002014
 accomplish negative sentiment is 0.036544010043144226
 accomplish neutral sentiment is 0.31196045875549316


In [48]:
from statistics import mean 
col_list = ['accomp1b', 'accomp2b']
def sentiForCol(col_list):
    pos_mean_ls = []
    for k in range(df.shape[0]):
        pos_ls = []
        for j in range(0, len(col_list)):
          text = df.iloc[k][col_list[j]]
          text = preprocess(text)
          encoded_input = tokenizer(text, return_tensors='pt')
          output = model(**encoded_input)
          scores = output[0][0].detach().numpy()
          scores = softmax(scores)
          ranking = np.argsort(scores)
          ranking = ranking[::-1]
          for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
        pos_mean = mean(pos_ls)
        pos_mean_ls.append(pos_mean)
    return pos_mean_ls
print(sentiForCol(col_list))
profile1 = sentiForCol(col_list)

[0.25089774, 0.36381948, 0.54198956, 0.25107777]


In [49]:
from statistics import mean 
col_list = ['accomp1b', 'accomp2b']
def sentiForCol(col_list):
    pos_mean_ls = []
    for k in range(df2.shape[0]):
        pos_ls = []
        for j in range(0, len(col_list)):
          text = df2.iloc[k][col_list[j]]
          text = preprocess(text)
          encoded_input = tokenizer(text, return_tensors='pt')
          output = model(**encoded_input)
          scores = output[0][0].detach().numpy()
          scores = softmax(scores)
          ranking = np.argsort(scores)
          ranking = ranking[::-1]
          for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
        pos_mean = mean(pos_ls)
        pos_mean_ls.append(pos_mean)
    return pos_mean_ls
print(sentiForCol(col_list))
profile2 = sentiForCol(col_list)

[0.63093585, 0.8839334, 0.6357721, 0.45534074]


In [50]:
df_profile1_interface_t_test = pd.DataFrame({'profile': 'profile1', 'interface_pos': profile1 })
df_profile2_interface_t_test = pd.DataFrame({'profile': 'profile2', 'interface_pos': profile2 })

In [52]:
df_profiles_combine_interface_t_test = pd.concat([df_profile1_interface_t_test, df_profile2_interface_t_test], axis=0)
df_profiles_combine_interface_t_test

,profile,interface_pos
0,profile1,0.250898
1,profile1,0.363819
2,profile1,0.541990
3,profile1,0.251078
0,profile2,0.630936
1,profile2,0.883933
2,profile2,0.635772
3,profile2,0.455341


In [54]:
from scipy import stats
from itertools import combinations
hobbies = df_profiles_combine_interface_t_test['profile'].unique()

# Perform pairwise t-tests
pairwise_results = []
for (hobby1, hobby2) in combinations(hobbies, 2):
    group1 = df_profiles_combine_interface_t_test[df_profiles_combine_interface_t_test['profile'] == hobby1]['interface_pos']
    group2 = df_profiles_combine_interface_t_test[df_profiles_combine_interface_t_test['profile'] == hobby2]['interface_pos']
    t_stat, p_value = stats.ttest_ind(group1, group2)
    pairwise_results.append(((hobby1, hobby2), t_stat, p_value))
    print(p_value)

# Print results
for (hobby1, hobby2), t_stat, p_value in pairwise_results:
    print(f"{hobby1} vs. {hobby2}: T-statistic = {t_stat:.2f}, P-value = {p_value:.4f}")


0.036492836017393934
profile1 vs. profile2: T-statistic = -2.68, P-value = 0.0365
